<a href="https://colab.research.google.com/github/uwa234/Data-Science-Projects/blob/master/NLP_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install bert-for-tf2 

     |████████████████████████████████| 40kB 4.2MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-cp36-none-any.whl size=30114 sha256=5ae36beedc8cf0938b06ca74d2782064850a68d7778881a99f23cc1599112f70
  Stored in directory: /root/.cache/pip/wheels/cf/3f/4d/79d7735015a5f523648df90d871ce8e89a7df8185f7703eeab
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=541c2549e9d5bef3dbe623d96f792be9a3799f7ac96f1f3f55cd454c61455d79
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=4f67f1ee1cd23c94ea7ca2b657e261f1a21d4bbc32df2a3a7d0657c6566768e2
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
!pip install sentencepiece

     |████████████████████████████████| 1.0MB 8.9MB/s 


In [0]:
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [6]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving sample_submission.csv to sample_submission.csv
Saving test.csv to test.csv
Saving train.csv to train.csv
User uploaded file "sample_submission.csv" with length 22746 bytes
User uploaded file "test.csv" with length 420783 bytes
User uploaded file "train.csv" with length 987712 bytes


In [7]:
#Training data
train = pd.read_csv('train.csv')
print('Training data shape: ', train.shape)
train.head()

Training data shape:  (7613, 5)


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
import re

test_str = train.loc[417, 'text']

def clean_text(text):
    text = re.sub(r'https?://\S+', '', text) # Remove link
    text = re.sub(r'#\w+', '', text) # Remove hashtag
    text = re.sub(r'@\w+', '', text) # Remove mentions
    text = re.sub(r'\n',' ', text) # Remove line breaks
    text = re.sub('\s+', ' ', text).strip() # Remove leading, trailing, and extra spaces
    return text

print("Original text: " + test_str)
print("Cleaned text: " + clean_text(test_str))

Original text: Arson suspect linked to 30 fires caught in Northern California http://t.co/mmGsyAHDzb
Cleaned text: Arson suspect linked to 30 fires caught in Northern California


In [9]:
# Testing data 
test = pd.read_csv('test.csv')
print('Testing data shape: ', test.shape)
test.head()

Testing data shape:  (3263, 4)


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [0]:
def find_hashtags(tweet):
    return " ".join([match.group(0)[1:] for match in re.finditer(r"#\w+", tweet)]) or 'no'

def find_mentions(tweet):
    return " ".join([match.group(0)[1:] for match in re.finditer(r"@\w+", tweet)]) or 'no'

def find_links(tweet):
    return " ".join([match.group(0)[:] for match in re.finditer(r"https?://\S+", tweet)]) or 'no'

def process_text(df):
    
    df['text_clean'] = df['text'].apply(lambda x: clean_text(x))
    df['hashtags'] = df['text'].apply(lambda x: find_hashtags(x))
    df['mentions'] = df['text'].apply(lambda x: find_mentions(x))
    df['links'] = df['text'].apply(lambda x: find_links(x))
    # df['hashtags'].fillna(value='no', inplace=True)
    # df['mentions'].fillna(value='no', inplace=True)
    
    return df
    
train = process_text(train)
test = process_text(test)

In [0]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [0]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2",
                            trainable=True)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [13]:
tokenizer.tokenize("don't be so judgmental")

['don', "'", 't', 'be', 'so', 'judgment', '##al']

In [0]:
train =train.fillna(' ')
test = test.fillna(' ')
train['text_final'] = train['text_clean']+' '+ train['keyword']+' '+ train['location']
test['text_final'] = test['text_clean']+' '+ test['keyword']+' '+ test['location']

In [0]:
train['lowered_text'] = train['text_final'].apply(lambda x: x.lower())
test['lowered_text'] = test['text_final'].apply(lambda x: x.lower())

In [0]:
train_input = bert_encode(train.lowered_text.values, tokenizer, max_len=320)
test_input = bert_encode(test.lowered_text.values, tokenizer, max_len=320)
train_labels = train.target.values

In [0]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

In [0]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    hidden1 = Dense(100, activation='relu')(clf_output)
    hidden2 = Dense(50, activation='relu')(hidden1)
    out = Dense(1, activation='sigmoid')(hidden2)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [19]:
model = build_model(bert_layer, max_len=320)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 320)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 320)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 320)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [21]:
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.1,
    epochs=10,
    batch_size=16
)

Epoch 1/10
429/429 [==============================] - 492s 1s/step - loss: 0.3746 - accuracy: 0.8419 - val_loss: 0.4187 - val_accuracy: 0.8241
Epoch 2/10
429/429 [==============================] - 492s 1s/step - loss: 0.3453 - accuracy: 0.8590 - val_loss: 0.4250 - val_accuracy: 0.8241
Epoch 3/10
429/429 [==============================] - 492s 1s/step - loss: 0.3203 - accuracy: 0.8726 - val_loss: 0.4485 - val_accuracy: 0.8045
Epoch 4/10
429/429 [==============================] - 492s 1s/step - loss: 0.2941 - accuracy: 0.8856 - val_loss: 0.4308 - val_accuracy: 0.8150
Epoch 5/10
429/429 [==============================] - 492s 1s/step - loss: 0.2718 - accuracy: 0.8921 - val_loss: 0.4550 - val_accuracy: 0.8045
Epoch 6/10
429/429 [==============================] - 492s 1s/step - loss: 0.2476 - accuracy: 0.9059 - val_loss: 0.4742 - val_accuracy: 0.8018
Epoch 7/10
429/429 [==============================] - 492s 1s/step - loss: 0.2273 - accuracy: 0.9133 - val_loss: 0.4877 - val_accuracy: 0.8018

In [0]:
model.save('bert100_50.h5')

In [23]:
print(train_history)

In [0]:
test_pred = model.predict(test_input)

In [0]:
#from sklearn.metrics import confusion_matrix, classification_report
#print(confusion_matrix(y_test, y_pred))
#print(classification_report(y_test, y_pred.round().astype(int)))

In [0]:
submission = pd.read_csv("sample_submission.csv")
submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)